# Introduction

[A écrire]

## Importation des Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co

import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
import matplotlib.pyplot as mplt



from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
import requests
from io import BytesIO

# Récupération et traitement des données

On a les bases : 
- `All streaming shows` (AS) trouvée sur kaggle
- `Netflix titles` trouvée sur kaggle
- `IMDB 250` scrapée sur le site éponyme : df_tvshow
- `df_carac` scrappée sur "https://www.the-numbers.com/home-market/netflix-daily-chart/2020/12/06"

On avait commencé à travailler sur `Netflix Titles` mais le manque de variables exploitables pour une modélisation nous a mené à chercher d'autres bases.<br>

On avait donc scrappé IMDB pour récupérer le TOP 250 des films et des séries (selon IMDB), cette base n'a finalement pas beaucoup servi les titres étant en français (impossible de trouver une version du site où les titres étaient en anglais).<br>

On a ensuite scrapper les données du site 'https://www.the-numbers.com/home-market/netflix-daily-chart/2020/12/06' qui recense quotidiennement depuis mi mars 2020 le Top 10 Netflix, donne quelques caractéristiques de ces films et séries et calcule un score d'audience : chaque jour à la première place donne 10 points, à la deuxième 9 etc...<br>

Nous nous sommes alors heurté à un problème de cohérence temporelle. En effet notre base originale avec les caractéristiques des séries (`Netflix Titles`) s'arrêtait en janvier 2020 et notre base des Top quotidien commencaient début 2020, la plupart des titres contenus dans cette dernière n'apparaissent donc pas dans la première.<br>

Nous avons donc cherché une autre base, plus récente. Pour cela nous avons du prendre une base plus large en termes de platforme mais avec seulement des séries. `All streaming shows` recense toutes les séries diffusées sur des plateformes.<br>

## `Net-tit`

In [2]:
#cellule Myrtille
Net_tit = pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\netflix_titles.csv')

In [3]:
#cellule léo
#Net_tit = pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\netflix_titles.csv')

In [4]:
#Net_tit.head(5)

On compte le nombre de valeurs manquantes pour `date_added` et `release_year`, les deux variables sur lesquelles on peut faire des opérations numériques (pour comparer les séries/films qui ont été créés par Netflix ou juste rachetés)


In [5]:
Net_tit[['date_added']].isna().sum() #11 valeurs manquantes
Net_tit[['release_year']].isna().sum() #0 valeurs manquantes

release_year    0
dtype: int64

On créé une variable `year_added` codée en int qui renseigne l'année où le film/série a été ajouté sur Netflix et on remplace les valeurs manquantes par des zéros. Pour cela on crée une regex qui nous permet d'extraire l'année de la date complète donnée dans date_added. On remplace ensuite les NaN par des 0 et on convertit les valeurs en entiers.

In [6]:
Net_tit[['date_added']].iloc[0:6233]

,date_added
0,"September 9, 2019"
1,"September 9, 2016"
2,"September 8, 2018"
3,"September 8, 2018"
4,"September 8, 2017"
...,...
6228,NaN
6229,NaN
6230,NaN
6231,NaN


In [7]:
regex = r'([0-2][0-9][0-9][0-9])'
Net_tit['year_added'] = Net_tit['date_added'].str.extract(regex,expand=True)
Net_tit['year_added'] = Net_tit['year_added'].fillna(0)
Net_tit['year_added'] = Net_tit['year_added'].astype(str).astype(int)

### Création de `df1`

On choisit des colonnes de Net_tit puis on ordonne par ordre décroissant de sortie. Pour cela on utilise date_added, plus précis que year_added.

In [8]:
df1 = Net_tit[['type', 'title','director', 'cast', 'date_added', 'year_added', 'release_year', 'duration', 'listed_in']]
df1 = df1.sort_values('release_year', ascending = False) #probleme si on veut classer selon year_added
df1.head(5)

,type,title,director,cast,date_added,year_added,release_year,duration,listed_in
3467,TV Show,Medical Police,NaN,"Erinn Hayes, Rob Huebel, Malin Akerman, Rob Co...","January 10, 2020",2020,2020,1 Season,"Crime TV Shows, TV Action & Adventure, TV Come..."
3249,Movie,All the Freckles in the World,Yibrán Asuad,"Hánssel Casillas, Loreto Peralta, Andrea Sutto...","January 3, 2020",2020,2020,90 min,"Comedies, International Movies, Romantic Movies"
3220,TV Show,Dracula,NaN,"Claes Bang, Dolly Wells, John Heffernan","January 4, 2020",2020,2020,1 Season,"British TV Shows, International TV Shows, TV D..."
3427,Movie,Leslie Jones: Time Machine,"David Benioff, D.B. Weiss",Leslie Jones,"January 14, 2020",2020,2020,66 min,Stand-Up Comedy
3436,TV Show,The Healing Powers of Dude,NaN,"Jace Chapman, Larisa Oleynik, Tom Everett Scot...","January 13, 2020",2020,2020,1 Season,"Kids' TV, TV Comedies, TV Dramas"


### Création de `df_movie` et `df_tvshow`

On sépare `df1` en deux data frames, l'un pour les séries, l'autre pour les films.

In [9]:
df1[['type']].isna().sum() #pas de valeurs manquantes

type    0
dtype: int64

In [10]:
df_tvshow = df1[df1['type'].isin(["TV Show"])]
df_movie = df1[df1['type'].isin(['Movie'])]

## Top 250 IMDB

### `recup_notes`

On crée une fonction qui met dans un data frame les notes des 250 mieux notés sur ImDB (films ou séries).

In [188]:
def recup_notes(url):

    """On récupère la page internet------------------"""
    from urllib import request
    request_text = request.urlopen(url).read()
    #print(request_text[:1000]) 
    page = bs4.BeautifulSoup(request_text, "lxml")
    #print(type(page))


    """On récupère les données------------------"""
    tableau=page.find("table",{'class' : 'chart full-width'})
    #print(tableau)
    table_body=tableau.find('tbody')
    rows = table_body.find_all('tr')
    #print (rows)


    """On récupére les élements correspondant au titre et à la note--------------"""
    new_rows=[] #on crée une liste vide dans laquelle on va mettre le contenu des lignes
    for row in rows:
        cols = row.find_all('td')
        #print(type(cols))
        new_cols = []
        for ele in cols:
            new_cols.append([ele.text.strip()])
        #print(new_cols)
        new_cols=new_cols[1:3]
        #new_cols_title=new_cols[0]
        new_cols[0]= new_cols[0][0]
        new_cols[1]= float(new_cols[1][0])
        new_rows.append(new_cols)
  #print (new_rows)

    """On nettoie les élements pour n'avoir que des listes de titres et notes"""
    """L'élement correspondant au titre est entouré d'informations parasites, on s'en
    débarasse avec 2 regex successives"""

    regex_debut = "[\d]?[\d]?[\d]\.[\n][\s]{6}"
    regex_fin="[\n]\([1-2][\d]{3}\)"

    clean_rows=copy.copy(new_rows)

    for i in range(len(new_rows)):
        a_suppr=re.findall(regex_debut,new_rows[i][0])
        #print (a_suppr[0])
        clean_rows[i][0]=new_rows[i][0].replace(a_suppr[0],"")
    #print (clean_rows)

    really_clean_rows=copy.copy(clean_rows)

    for i in range(len(clean_rows)):
        a_suppr_fin=re.findall(regex_fin,clean_rows[i][0])
        #print (a_suppr_fin[0])
        really_clean_rows[i][0]=clean_rows[i][0].replace(a_suppr_fin[0],"")
    #print (really_clean_rows)

    """On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    La clé est le titre, la valeur la note."""
    dico_top = dict()
    for row in really_clean_rows:
        cols=row
        dico_top[cols[0]] = cols[1:]
    dico_top

    """On crée le dataframe"""
    df_top = pd.DataFrame.from_dict(dico_top,orient='index')
  

    """"On rajoute une index différente du titre"""
    df_top.reset_index(level=0, inplace=True)

    """On change le nom des colonnes"""
    df_top.rename(columns={'index': 'Title', 0: 'IMDb rating'}, inplace=True)

    #print(df_top)
    return (df_top)


### Création de `df_top_shows` et `df_top_movies`

In [189]:
url_top_shows = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"
url_top_movies= "https://www.imdb.com/chart/top/?ref_=nv_mv_250"

In [13]:
df_top_shows=recup_notes(url_top_shows)
df_top_shows.head()

,Title,IMDb rating
0,Planet Earth II,9.5
1,Planète Terre,9.4
2,Frères d'armes,9.4
3,Breaking Bad,9.4
4,Chernobyl,9.4


In [14]:
df_top_movies=recup_notes(url_top_movies)
df_top_movies.head()

,Title,IMDb rating
0,Les évadés,9.2
1,Le parrain,9.1
2,"Le parrain, 2ème partie",9.0
3,The Dark Knight: Le chevalier noir,9.0
4,12 hommes en colère,8.9


On sauvegarde ces deux df dans deux fichiers csv

In [15]:
a=df_top_movies.to_csv("df_top_movies.csv", index = False)
a=df_top_shows.to_csv("df_top_shows.csv", index = False)

## Top Netflix quotidien

De mi mars à début décembre on a 252 top quotidiens d'environ 10 lignes. On a choisi de les concaténer par mois. Comme cette opération prend un certain temps nous avons enregistrer les df obtenus sous forme de csv pour éviter de devoir répéter l'opération et les opérations de concaténations sont en annexe sous forme de commentaire (pour éviter qu'elle se lancent lors d'un Run All).

### `recup_top_netflix_gen`

Pour scrapper ces 252 top quotidiens on a défini une fonction qui prend comme argument la date souhaitée *(aaaa/mm/jj)* et en argument conditionnel le début de l'url.<br>La fonction renvoie le df correspondant.

In [16]:
def recup_top_netflix_gen(date,url="https://www.the-numbers.com/home-market/netflix-daily-chart/"):
    
    url_enrichi = url+date
    from urllib.request import Request, urlopen
    req_text = urlopen(Request(url_enrichi, headers={'User-Agent': 'Mozilla/5.0'})).read()
    page = bs4.BeautifulSoup(req_text, "lxml")
    #print (page)

    """On récupère les données------------------"""
    #tableau=page.find("table",{'class' : 'chart full-width'})
    tableau=page.find("div",{'id':'page_filling_chart'})
    #print(tableau)
    #table_body=tableau.find('tbody')
    #print(table_body)
    rows = tableau.find_all('tr')
    #print (rows)

  
    """On récupére tous les élements correspondant au titre et à la note et On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    L'index est le titre, la valeur la note.
    --------------"""
    new_rows=[]#on crée une liste vide dans laquelle on va mettre le contenu des lignes
    new_cols = 10*[[]]
    dico_top_netflix = dict()
    i = 0
    for row in rows:
        cols = row.find_all('td') # c'est une liste
        cols = [ele.text.strip() for ele in cols]
        #print (cols)
        if len(cols)> 0:
            dico_top_netflix[cols[0]] = cols[1:]
    #print(dico_top_netflix)    

    """"""

    """On crée ENFIN le dataframe"""
    df_top_netflix_gen = pd.DataFrame.from_dict(dico_top_netflix,orient='index')

    """On rajoute une index différente du titre"""
    for row in rows:
        cols = row.find_all('th')
        #print(cols)
        if len(cols) > 0 : 
            cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
            columns_title = cols
    #print (columns_title)
    

    """On change le nom des colonnes"""
    df_top_netflix_gen.columns=columns_title[1:]
    
    """On vire les colonnes qui ne nous intéresse pas"""
    del(df_top_netflix_gen['Yd'])
    del(df_top_netflix_gen['Lw'])
    del(df_top_netflix_gen['Watch Now'])
    
    
    """On rajoute une colonne 'Measure Date' """
    df_top_netflix_gen['Measure Date']= [date]*df_top_netflix_gen.shape[0]
    #print ("Graph au "+ date)
    return (df_top_netflix_gen)

Le Top10 Netflix aux US le 13 décembre 2020.

In [17]:
df_12_13=recup_top_netflix_gen("2020/12/13")
df_12_13

,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
1,Ava,Movie,,,6,50,2020/12/13
2,The Prom,Movie,Yes,"Dec 11, 2020",2,15,2020/12/13
3,Manhunt,TV Show,,,4,29,2020/12/13
4,How the Grinch Stole Christmas,Movie,,,14,36,2020/12/13
5,Triple 9,Movie,,,5,28,2020/12/13
6,Selena: The Series,TV Show,Yes,"Dec 4, 2020",9,81,2020/12/13
7,Big Mouth,TV Show,Yes,"Sep 29, 2017",9,74,2020/12/13
8,Virgin River,TV Show,Yes,"Nov 27, 2020",16,128,2020/12/13
9,The Christmas Chronicles 2,Movie,Yes,"Nov 25, 2020",13,90,2020/12/13
10,The Queen's Gambit,TV Show,Yes,"Oct 23, 2020",51,392,2020/12/13


### Création des df des mois et concaténation vers `df_carac`

On crée d'abord une liste de toutes les dates avec un Top 10.

Pour cela on crée une liste avec toutes les dates de 2020 puis on enlève celles pour lesquelles on a pas de Top. On compte bien 252 dates valides.

In [18]:
liste_dates_valides=[]
for m in range (3,13):
    m=str(m)
    if len(m)==1:
        m = "0"+m
    for d in range (1,32):
        d=str(d)
        if len(d)==1:
            d="0"+d
        date = "2020/"+m+"/"+d
        liste_dates_valides.append(date)

#On supprime les dates sans valeurs
for i in ['2020/03/01', '2020/03/02', '2020/03/03', '2020/03/04', '2020/03/05', '2020/03/06', '2020/03/07', '2020/03/08', 
          '2020/03/09', '2020/03/10', '2020/03/11', '2020/03/12', '2020/03/13', '2020/03/14', '2020/03/15', '2020/03/16', 
          '2020/03/17', '2020/03/18', '2020/03/19', '2020/03/20', '2020/03/21', '2020/03/22', '2020/03/23', '2020/03/28', 
          '2020/03/29', '2020/04/04', '2020/04/05', '2020/04/11', '2020/04/12', '2020/04/18', '2020/04/19', '2020/04/25', 
          '2020/04/26', '2020/08/09', '2020/08/30', '2020/12/14', '2020/12/15', '2020/12/16', '2020/12/17', '2020/12/18',
          '2020/12/19', '2020/12/20', '2020/12/21', '2020/12/22', '2020/12/23', '2020/12/24', '2020/12/25', '2020/12/26', 
          '2020/12/27', '2020/12/28', '2020/12/29', '2020/12/30', '2020/12/31', '2020/04/31', '2020/06/31', '2020/09/31', 
          '2020/11/31', '2020/08/06']:
    liste_dates_valides.remove(i)
print ('On a ',len (liste_dates_valides), ' dates valides.')

On a  252  dates valides.


On crée un string avec toutes les dates séparées par une virgule.

In [19]:
dates=""
for i in liste_dates_valides:
    dates=dates+i+','
dates=dates[:-1]

On crée une liste avec pour éléments les listes des dates valides de chaque mois. On en fait un dictionnaire : `dico_mois`.

In [20]:
janvier = []
fevrier = []
mars=[]
avril=[]
mai=[]
juin=[]
juillet=[]
aout=[]
septembre=[]
octobre=[]
novembre=[]
decembre=[]
mois = [[],janvier,fevrier,mars,avril,mai,juin,juillet,aout,septembre,octobre,novembre,decembre]
for i,moi in enumerate(mois):
    
    if i <10: regex = re.compile("([2][0][2][0]/[0]["+str(i)+"]/[0-3][0-9])")
    else : regex = re.compile("([2][0][2][0]/[1]["+str(i-10)+"]/[0-3][0-9])")
    mois[i].append(regex.findall(dates))
mois=mois[3:]

dico_mois={'mars': mars, 'avril': avril, 'mai': mai, 'juin': juin, 'juillet': juillet,'aout': aout,
           'septembre': septembre, 'octobre':octobre, 'novembre': novembre, 'decembre': decembre}

### Concaténation

On crée `dico_df_mois` qui contiendra le df de chaque mois, on l'initie en mettant le premier df de chaque mois.

In [21]:
df_mars = recup_top_netflix_gen("2020/03/24")
df_avril = recup_top_netflix_gen("2020/04/01")
df_mai = recup_top_netflix_gen("2020/05/01")
df_juin = recup_top_netflix_gen("2020/06/01")
df_juillet = recup_top_netflix_gen("2020/07/01")
df_aout = recup_top_netflix_gen("2020/08/01")
df_septembre = recup_top_netflix_gen("2020/09/01")
df_octobre = recup_top_netflix_gen("2020/10/01")
df_novembre = recup_top_netflix_gen("2020/11/01")
df_decembre = recup_top_netflix_gen("2020/11/01")

dico_df_mois={"mars" : df_mars, "avril" : df_avril, "mai":df_mai , "juin":df_juin , "juillet":df_juillet,
              "aout":df_aout ,"septembre" :df_septembre ,"octobre": df_octobre, "novembre" :df_novembre,"decembre": df_decembre}

On enrichit chaque df avec les autres jours du mois.

Cette cellule est longue à éxécuter : on la neutralise et on store les df obtenus dans des csv que nous utiliserons ensuite.

In [22]:
#for key,value in dico_mois.items():
#    print (key)
#    dico_df_mois[key] = recup_top_netflix_gen(value[0][0])
#    a_conc=0
#    for i in value[0][1:]:
#        a_conc = recup_top_netflix_gen(i)
#        dico_df_mois[key] = pd.concat([dico_df_mois[key],a_conc])
#        #print(dico_df_mois[key])


In [23]:
#for moi, df_moi in dico_df_mois.items():
#    a=df_moi.to_csv("df_"+moi+".csv", index = False) 

On récupère les CSV correspondants.

In [24]:
#Cellule Myrtille
df_mars=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_mars.csv')
df_avril=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_avril.csv')
df_mai=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_mai.csv')
df_juin=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_juin.csv')
df_juillet=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_juillet.csv')
df_aout=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_aout.csv')
df_septembre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_septembre.csv')
df_octobre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_octobre.csv')
df_novembre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_novembre.csv')
df_decembre=pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\df_decembre.csv')

In [26]:
#Cellule Léo
#df_mars=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_mars.csv')
#df_avril=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_avril.csv')
#df_mai=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_mai.csv')
#df_juin=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_juin.csv')
#df_juillet=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_juillet.csv')
#df_aout=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_aout.csv')
#df_septembre=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_septembre.csv')
#df_octobre=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_octobre.csv')
#df_novembre=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_novembre.csv')
#df_decembre=pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\df_decembre.csv')

On remarque que la construction du `Score` fait qu'il est strictement croissant pour une fonction donnée.
On va donc ordonner le df d'abord par titre puis par date décroissante. Ainsi la première occurence d'un titre donne son score le plus récent (et donc maximal), pour un mois donné.


On crée une fonction `sortby` qui ordonne les df de cette manière.

In [27]:
def sortby(df):
    return df.sort_values(['Title','Measure Date'],ascending=(True,False))
#sortby(df_mars)

In [28]:
sortby(df_decembre).head(10)

,Title,Type,Netflix Excl.,Netflix Release Date,Days In Top 10,Viewer- Ship Score,Measure Date
68,Alien Worlds,TV Show,Yes,"Dec 2, 2020",5,17,2020/12/07
58,Alien Worlds,TV Show,Yes,"Dec 2, 2020",4,15,2020/12/06
47,Alien Worlds,TV Show,Yes,"Dec 2, 2020",3,13,2020/12/05
33,Alien Worlds,TV Show,Yes,"Dec 2, 2020",2,10,2020/12/04
27,Alien Worlds,TV Show,Yes,"Dec 2, 2020",1,3,2020/12/03
120,Ava,Movie,NaN,NaN,6,50,2020/12/13
110,Ava,Movie,NaN,NaN,5,40,2020/12/12
100,Ava,Movie,NaN,NaN,4,30,2020/12/11
92,Ava,Movie,NaN,NaN,3,20,2020/12/10
82,Ava,Movie,NaN,NaN,2,12,2020/12/09


On crée `dico_carac` où chaque entrée est un titre différent et les valeurs sont le type, l'exclusivité Netflix et le Score.
On utilise les df dans le sens anti-chronologique pour récupérer le score le plus récent.

In [35]:
liste_df = [df_decembre,df_novembre,df_octobre,df_septembre,df_aout,df_juillet,df_juin,df_mai,df_avril,df_mars]
dico_carac=dict()

for df_moi in liste_df:
    for i in range(sortby(df_moi).shape[0]):
        if sortby(df_moi)['Title'][i] not in dico_carac:
            dico_carac[sortby(df_moi)['Title'][i]]=[sortby(df_moi)['Type'][i],sortby(df_moi)['Netflix Excl.'][i],sortby(df_moi)['Viewer- Ship Score'][i]]

#dico_carac

### Création de `df_carac`
On crée le df correspondant

In [90]:
df_carac = pd.DataFrame.from_dict(dico_carac,orient='index')
df_carac.columns=['Type','Netflix Exclusive','Score']
df_carac= df_carac.sort_values('Score',ascending=False)
df_carac

,Type,Netflix Exclusive,Score
The Queen's Gambit,TV Show,Yes,340
Outer Banks,TV Show,Yes,259
The Umbrella Academy,TV Show,Yes,231
365 Dni,Movie,NaN,198
Avatar the Last Airbender,TV Show,NaN,196
...,...,...,...
How to Train Your Dragon 2,Movie,NaN,1
Legacies,TV Show,NaN,1
Community,TV Show,NaN,1
The Christmas Chronicles,Movie,Yes,1


On remplit les valeurs manquantes de Netflix Exclusive par `'No'`. 

En effet, sur le site original la variable ne prend comme modalité que 'Yes' ou rien, on considère que rien veut dire que la série n'est pas une série originale Netflix. 

In [285]:
df_carac['Netflix Exclusive']=df_carac['Netflix Exclusive'].fillna('No')

On reset l'index et renomme la colonne `'Title'`.

In [91]:
df_carac=df_carac.reset_index()
df_carac.columns=['Title','Type','Netflix Exclusive','Score']

In [286]:
df_carac.head()

,Title,Type,Netflix Exclusive,Score,Above Mean
0,The Queen's Gambit,TV Show,Yes,340,True
1,Outer Banks,TV Show,Yes,259,True
2,The Umbrella Academy,TV Show,Yes,231,True
3,365 Dni,Movie,No,198,True
4,Avatar the Last Airbender,TV Show,No,196,True


On store ce df sous un csv du même nom.

In [287]:
gfg_csv_data = df_carac.to_csv('df_carac.csv', index = False)

## All Streaming Shows (`AS`)

All Streaming Shows est une base trouvée sur Kaggle qui recense les Séries (uniquement) diffusées en 2020.

Elle présente l'avantage de contenir plusieurs valeurs exploitables en particulier : le genre, la plateforme de diffusion et les notes IMDB et Rotten Tomatoes.

In [39]:
#Cellule léo
#AS= pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\All_Streaming_Shows.csv',sep=',')
#AS.shape[0] #12 353
#AS.head(5)

In [44]:
#Cellule Myrtille
AS= pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\All_Streaming_Shows.csv',sep=',')
AS.shape[0] #12 353
AS.head(5)

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video


### Nettoyage préliminaire et ajout de `Top IMDB`

On nettoie la colonne `No of Seasons` pour ne conserver que le nombre.

Pour cela on utilise une regex.

In [45]:
regex = r'([0-9]?[0-9])'
AS['No of Seasons'] = AS['No of Seasons'].str.extract(regex,expand=True)
AS['No of Seasons'] = AS['No of Seasons'].fillna(0)
AS['No of Seasons'] = AS['No of Seasons'].astype(str).astype(int)

In [46]:
AS.head()

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5,Netflix
1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8,"HBO MAX,HBO"
2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4,"Free Services,HBO MAX,Hulu"
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3,Netflix
4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2,Prime Video


On enrichi ensuite cette base d'une variable binaire 'Top IMDB' qui prend 1 comme valeur quand la série figure dans le Top IMDB, 0 sinon.

Nous avons codé cette variable en observant dans notre base Top IMDB 250 qu'une note supérieure à 8,4 valait une inscription dans le Top.

(Il n'était pas possible de directement fusioner les deux bases à cause de l'incompatibilité des titres :en francais pour Top IMDB et anglais ici.)

In [48]:
df_top_shows.tail()

,Title,IMDb rating
239,Community,8.4
240,"Unsere Mütter, unsere Väter",8.4
241,Vikings,8.4
242,Clannad: After Story,8.4
243,Feud,8.4


In [49]:
AS['Top IMDB'] = AS['IMDB Rating'].apply(lambda x: '1' if x >= 8.4 else '0')
AS['Top IMDB'] = AS['Top IMDB'].astype(str).astype(int)

In [191]:
AS.sort_values('IMDB Rating',ascending=False)[745:760]

### Création de sous bases propres aux plateformes `AS_N`, `AS_P`, `AS_H`, `AS_D`, `AS_F`

On aura donc une base par plateforme pour : Netflix, Prime Video, Hulu, Disney+ et fuboTV.

L'inconvénient de cette façon et de le faire est qu'on exclue les titres où plusieurs plateformes sont renseignées.

On peut considérer que ce sont des bases avec seulement les "exclusivités" de chaque plateforme.

In [571]:
AS['Streaming Platform'].isin(['Netflix']).sum() #1 427 titres Netflix
AS_N=AS[AS['Streaming Platform'].isin(['Netflix'])]
AS_P=AS[AS['Streaming Platform'].isin(['Prime Video'])]
AS_H=AS[AS['Streaming Platform'].isin(['Hulu'])]
AS_D=AS[AS['Streaming Platform'].isin(['Disney+'])]
AS_F=AS[AS['Streaming Platform'].isin(['fuboTV'])]

gfg_csv_data = AS_N.to_csv('AS_N.csv', index = False)

In [65]:
print ('On a ',AS['Streaming Platform'].isin(['Netflix']).sum()," titres pour Netflix, ",
       AS['Streaming Platform'].isin(['Prime Video']).sum()," pour Prime Video, ",
      AS['Streaming Platform'].isin(['Hulu']).sum(), " pour Hulu et ",
      AS['Streaming Platform'].isin(['fuboTV']).sum()," pour fuboTV.")

On a  1427  titres pour Netflix,  1184  pour Prime Video,  665  pour Hulu et  298  pour fuboTV.


## Jointures

On effectue les jointures suivantes :

- **df_tvshow x cara** = `OGTV_x_carac`

- **df_movie x carac** = `OGM_x_carac`

- **AS x carac**

- **AS_N x carac**

In [99]:
OGTV_x_carac=df_tvshow.set_index('title').join(df_carac.set_index('Title'))
OGTV_x_carac.rename(columns={'index': 'Title'}, inplace=True)
gfg_csv_data = OGTV_x_carac.to_csv('OGTV_x_carac.csv', index = False)

OGM_x_carac=df_movie.set_index('title').join(df_carac.set_index('Title'))
OGM_x_carac.rename(columns={'index': 'Title'}, inplace=True)
gfg_csv_data = OGM_x_carac.to_csv('OGM_x_carac.csv', index = False) 

In [103]:
OGTV_x_carac.sort_values('Score',ascending=False).head()

,type,director,cast,date_added,year_added,release_year,duration,listed_in,Type,Netflix Exclusive,Score
The Umbrella Academy,TV Show,NaN,NaN,"February 15, 2019",2019,2019,1 Season,"TV Action & Adventure, TV Mysteries, TV Sci-Fi...",TV Show,Yes,231.0
Dead to Me,TV Show,NaN,"Christina Applegate, Linda Cardellini, James M...","May 3, 2019",2019,2019,1 Season,"TV Comedies, TV Dramas",TV Show,Yes,175.0
Ozark,TV Show,NaN,"Jason Bateman, Sofia Hublitz, Laura Linney, Sk...","August 31, 2018",2018,2018,2 Seasons,"Crime TV Shows, TV Dramas, TV Thrillers",TV Show,Yes,169.0
The Crown,TV Show,NaN,"Claire Foy, John Lithgow, Matt Smith, Vanessa ...","November 17, 2019",2019,2019,3 Seasons,"British TV Shows, International TV Shows, TV D...",TV Show,Yes,138.0
Riverdale,TV Show,Rob Seidenglanz,"K.J. Apa, Lili Reinhart, Camila Mendes, Cole S...","May 23, 2019",2019,2019,3 Seasons,"Crime TV Shows, TV Dramas, TV Mysteries",TV Show,NaN,103.0


In [107]:
print ("On a seulement ",OGTV_x_carac['Score'].notna().sum()," scores, c'est très peu, d'où l'utilisation de AS et ses dérivées.")

On a seulement  45  scores, c'est très peu, d'où l'utilisation de AS et ses dérivées.


In [116]:
AS_x_carac = AS.set_index('Series Title').join(df_carac.set_index('Title'))
gfg_csv_data = AS_x_carac.to_csv('AS_x_carac.csv', index = False) 

In [117]:
print ("On a ",AS_x_carac['Score'].notna().sum()," scores, ça reste peu, mais c'est un peu mieux.")

On a  118  scores, ça reste peu, mais c'est un peu mieux.


In [110]:
AS_N_x_carac=AS_N.set_index('Series Title').join(df_carac.set_index('Title'))
AS_N_x_carac.rename(columns={'Unnamed: 0': 'Title'}, inplace=True)
gfg_csv_data = AS_N_x_carac.to_csv('AS_N_x_carac.csv', index = False) 

In [118]:
print ("Pour Netflix, on en a ",AS_N_x_carac['Score'].notna().sum())

Pour Netflix, on en a  73


# Analyse descriptive et représentation graphique

## Analyse des genres dans les premières bases Netflix : `df_tvshow` et `df_movie`

### Films v. Séries : `count_genra`
Fonction qui renvoie le nombre d'occurences de chaque genre ce qui permet de tracer les effectifs des genres les plus populaires.

In [275]:
def count_genra (data_f,nombre=22,inv=0,internat=1):
    
    "---------------On récupère les catégories---------------"
    col = "listed_in"
    categories = ", ".join(data_f['listed_in']).split(", ")
    #print (categories)

    if internat != 1:
        while "International TV Shows" in categories:
            categories.remove("International TV Shows")
    while 'International Movies' in categories:
        categories.remove('International Movies')

    #print(categories)
    
    "-----On compte le nombre d'occurences de chaque type et on garde les 'nombre' qui apparaissent le plus souvent-----"
    counter_list = co.Counter(categories).most_common(nombre)
    #print (type(counter_list_all))
    
    "-----On conserve le premier élément de chaque couple de counter_list (ie. sur le genre)-----"
    #On conserve le premier élément de chaque couple de counter_list (ie. sur le genre)
    if inv == 0: labels = [_[0] for _ in counter_list]
    else : labels = [_[0] for _ in counter_list][::-1]
    #print (labels)

    "-----Idem pour le nombre d'occurences-----"
    if inv == 0: values = [_[1] for _ in counter_list]
    else : values = [_[1] for _ in counter_list][::-1]
    
    return(labels,values)    

On peut donc comparer les genres les plus populaires et leurs effectifs pour les films et les séries.

In [276]:
fig = make_subplots(rows=1, cols=2,shared_yaxes=True)

fig.add_trace(go.Bar(
    y=count_genra(df_movie,5)[1], x=count_genra(df_movie,5)[0], 
    orientation = 'v', name='Films', marker_color='yellow'),
             row = 1, col =1)

fig.add_trace(go.Bar(
    y=count_genra(df_tvshow,5)[1], x=count_genra(df_tvshow,5)[0],
    orientation = 'v', name='Séries', marker_color='#58FA82'),
             row =1, col=2)

fig.update_layout( template = 'presentation', margin=dict(l=50, r=50, t=60, b=120), title = "Effectifs des Genres")

fig.update_layout(legend=dict( yanchor="top", y=1, xanchor="left", x=0.85))

fig.show()

### Dans le temps

On remarque que les séries sont enregistrées en tant que leur dernière saison sortie. Par exemple, la série The Crown est enregistré comme un seul titre de 3 saisons sorti en 2019. L'analyse dans le temps n'est donc pas pertinente pour les séries, les dates étant tirées vers le présent par construction.

On va donc se concentrer, pour cette partie, sur les films.

On remarque que `year_added` prend ses valeurs entre 2008 et 2020.

In [202]:
df_movie.sort_values("year_added")

,type,title,director,cast,date_added,year_added,release_year,duration,listed_in
6231,Movie,Little Baby Bum: Nursery Rhyme Friends,NaN,NaN,NaN,0,2016,60 min,Movies
3742,Movie,To and From New York,Sorin Dan Mihalcescu,"Barbara King, Shaana Diya, John Krisiukenas, Y...","January 1, 2008",2008,2006,81 min,"Dramas, Independent Movies, Thrillers"
1237,Movie,Splatter,Joe Dante,"Corey Feldman, Tony Todd, Tara Leigh, Erin Way...","November 18, 2009",2009,2009,29 min,Horror Movies
1590,Movie,Just Another Love Story,Ole Bornedal,"Anders W. Berthelsen, Rebecka Hemse, Nikolaj L...","May 5, 2009",2009,2007,104 min,"Dramas, International Movies"
1543,Movie,Mad Ron's Prevues from Hell,Jim Monaco,"Nick Pawlow, Jordu Schell, Jay Kushwara, Micha...","November 1, 2010",2010,1987,84 min,"Cult Movies, Horror Movies"
...,...,...,...,...,...,...,...,...,...
3506,Movie,Dragonheart: A New Beginning,Doug Lefler,"Christopher Masterson, Harry Van Gorkum, Rona ...","January 1, 2020",2020,2000,84 min,"Action & Adventure, Sci-Fi & Fantasy"
3383,Movie,Where the Money Is,Marek Kanievska,"Paul Newman, Linda Fiorentino, Dermot Mulroney...","January 15, 2020",2020,2000,89 min,"Action & Adventure, Comedies, Dramas"
3445,Movie,Betty White: First Lady of Television,Steve Boettcher,"Betty White, Valerie Bertinelli, Georgia Engel...","January 12, 2020",2020,2018,56 min,Documentaries
3564,Movie,Stuart Little,Rob Minkoff,"Michael J. Fox, Jonathan Lipnicki, Geena Davis...","January 1, 2020",2020,1999,85 min,"Children & Family Movies, Comedies"


On crée une fonction qui sort les genres les plus populaires pour 3 années données.

In [280]:
def genra_time(year1,year2,year3,nb=3):
    df1=df_movie[df_movie['year_added'].isin([year1])]
    df2=df_movie[df_movie['year_added'].isin([year2])]
    df3=df_movie[df_movie['year_added'].isin([year3])]
    
    fig = make_subplots(rows=3, cols=1,shared_xaxes=True,vertical_spacing=0.02)

    fig.add_trace(go.Bar(
        y=count_genra(df1,nb,1)[0], x=count_genra(df1,nb,1)[1], 
        orientation = 'h', name=str(year1), marker_color='yellow'),
                 row = 1, col =1)

    fig.add_trace(go.Bar(
        y=count_genra(df2,nb,1)[0], x=count_genra(df2,nb,1)[1],
        orientation = 'h', name=str(year2), marker_color='#58FA82'),
                 row =2, col=1)
    
    fig.add_trace(go.Bar(
        y=count_genra(df3,nb,1)[0], x=count_genra(df3,nb,1)[1],
        orientation = 'h', name=str(year3), marker_color='red'),
                 row =3, col=1)

    fig.update_layout( template = 'presentation', margin=dict(l=230, r=60, t=60, b=30),
                      title = "Effectifs des Genres dans le Temps", width=900, height=600,
                      legend=dict( yanchor="top", y=1, xanchor="left", x=0.78))

    fig.show()
    return

In [279]:
genra_time(2017,2018,2019,6)

## Succès du contenu original Netflix avec `df_carcac`

Le contenu original Netflix a pour réputation d'être de moins bonne qualité que le reste du contenu de la plateforme, cependant il est aggressivement mis en avant.

On se demande si le contenu original a un meilleur score de viewership que le reste du contenu présent sur la plateforme.
Bien sûr, ici l'échantillon n'est pas représentatif de l'ensemble du contenu de la plateforme, on n'a que le contenu relativement à succès. On regarde, comment le contenu *à succès* se répartit entre le contenu original et le reste.

On compare d'abord le nombre de titres originaux au score supérieur au score moyen au nombre des autres titres remplissant la même condition. <br> Pour cela on crée une nouvelle colonne `Above Mean` de booléens. On reçoit un message de `Warning` parce qu'on essaie de changer la copie d'un autre data frame.

In [292]:
df_carac[55:65]

,Title,Type,Netflix Exclusive,Score,Above Mean
55,Space Force,TV Show,Yes,30,True
56,Bad Teacher,Movie,No,29,True
57,The Good Place,TV Show,No,29,True
58,History 101,TV Show,Yes,28,True
59,Blood of Zeus,TV Show,Yes,24,True
60,Last Chance U,TV Show,Yes,23,True
61,Love,Movie,No,23,True
62,Blood Father,Movie,No,21,False
63,Crazy Delicious,TV Show,Yes,20,False
64,I Am a Killer: Released,TV Show,Yes,20,False


In [328]:
mean_all = df_carac['Score'].mean()

df_carac["Above Mean"]=0
for i in range (df_carac.shape[0]):
    if df_carac['Score'][i] >= mean_all:
        df_carac['Above Mean'][i]=True
    else :
        df_carac['Above Mean'][i]=False

'mean_all = df_carac[\'Score\'].mean()\n\ndf_carac["Above Mean"]=0\nfor i in range (df_carac.shape[0]):\n    if df_carac[\'Score\'][i] >= mean_all:\n        df_carac[\'Above Mean\'][i]=True\n    else :\n        df_carac[\'Above Mean\'][i]=False'

On crée un data frame donnant ces effectifs.

In [295]:
dico={'Netflix Exclusive': [df_carac['Netflix Exclusive'].isin(['Yes']).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'Yes']['Above Mean'].isin([True]).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'Yes']['Above Mean'].isin([False]).sum()],
     'Other': [df_carac['Netflix Exclusive'].isin(['No']).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'No']['Above Mean'].isin([True]).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'No']['Above Mean'].isin([False]).sum()]}

df_graph=pd.DataFrame.from_dict(dico,orient='index')
df_graph.columns=['Effectif','Above Mean','Below Mean']
df_graph.reset_index()
df_graph

,Effectif,Above Mean,Below Mean
Netflix Exclusive,175,45,130
Other,116,17,99


In [295]:
dico={'Netflix Exclusive': [df_carac['Netflix Exclusive'].isin(['Yes']).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'Yes']['Above Mean'].isin([True]).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'Yes']['Above Mean'].isin([False]).sum()],
     'Other': [df_carac['Netflix Exclusive'].isin(['No']).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'No']['Above Mean'].isin([True]).sum(),
                            df_carac[df_carac['Netflix Exclusive'] == 'No']['Above Mean'].isin([False]).sum()]}

df_graph=pd.DataFrame.from_dict(dico,orient='index')
df_graph.columns=['Effectif','Above Mean','Below Mean']
df_graph.reset_index()
df_graph

,Effectif,Above Mean,Below Mean
Netflix Exclusive,175,45,130
Other,116,17,99


On trace des stacked bar plots.

In [298]:
fig = px.bar(df_graph, x=df_graph.index, 
             y=['Above Mean','Below Mean'])

fig.update_layout(template = 'presentation',margin=dict(l=80, r=80, t=60, b=30),
                  title = { 'text' : '<b>Succès du contenu original Netflix</b>', 'x' : 0.5},
                  xaxis_title="Type",
                  yaxis_title="Effectifs",
                  font_family = 'Calibri',title_font_color= 'black',
                  legend_title="Comparaison au score moyen",
                  legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.8),
                  showlegend = True)
fig.show()

Netflix a en valeur absolue et relativement plus de titres au dessus de la moyenne globale.

## Gneugneu `AS` et ses dérivées

### Années de création des séries sur Netflix


In [345]:
AS_N.head()

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform,Top IMDB
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5,Netflix,1
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3,Netflix,1
5,Dark,2017,16+,8.8,95,"Crime,Drama",A missing child causes four families to help e...,3,Netflix,1
7,Sherlock,2010,16+,9.1,94,"Action & Adventure,Crime",A modern update finds the famous sleuth and hi...,4,Netflix,1
14,Better Call Saul,2015,18+,8.7,92,"Comedy,Crime",Six years before Saul Goodman meets Walter Whi...,5,Netflix,1


`Year Released` code l'année de sortie et non de diffusion sur Netflix, en effet Netflix diffuse aussi des séries sorties bien avant leur début sur Netflix, par exemple : Friends. <br>
Cela explique qu'on ait des séries jusque dans les années 70, elles ont bien sur été diffusées bien après sur Netflix.
Malheuresement, dans cette base cette variable est absente.


- `sorties_an` compte donc le nombre de séries sorties (et non diffusées sur Netflix) pour une année donnée.
- `somme_sorties` donne la somme cummulée de ces effectifs


In [367]:
sorties_an = AS_N['Year Released'].value_counts().sort_index()
somme_sorties = AS_N['Year Released'].value_counts().sort_index().cumsum()

trace1 = go.Bar(x = list(sorties_an.index), y = list(sorties_an.values),name = 'Nombre de Séries sorties chaque année',
                marker_color='#58FA82')
trace2 = go.Scatter(x = list(somme_sorties.index), 
                    y = list(somme_sorties.values), 
                    mode='lines',name = 'Trendline du nombre de séries', marker_color='yellow')

fig = plt.subplots.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1, secondary_y=False)
fig.add_trace(trace2, secondary_y=True)

fig.update_layout(template = 'presentation',margin=dict(l=80, r=80, t=60, b=30),
                  title = { 'text' : '<b>Années de sortie des Séries</b>', 'x' : 0.5},
                 font_family = 'Calibri',title_font_color= 'black', showlegend = True)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))

fig.show()

### Analyse de la répartition des genres sur Netflix
On va d'abord représenter la répartition des genres sur Netflix.

Pour cela on utilise :

- `recup_cat` qui retourne une liste de tous les genres pertinents qui apparaissent dans le data frame donné comme argument

- `count_genra` qui compte les effectifs de chaque genre et retourne les listes extraites un dictionnaire des genres les plus populaires et leurs effectifs (le nombre de genre est défini par l'argument conditionnel nombre)

- `trace` qui trace le graph batons correspondant à des effectifs de genre en prenant comme argument une liste de genres, un liste de leurs effectifs et un argument contionnel pour le titre.
          

In [369]:
"""-------------------------Fonction qui récupère les genres des Séries-------------------------"""

def recup_cat(data):
    
    col = "Genre"
    categories = ", ".join(data['Genre']).split(", ")
    #print (categories)
    categories = ",".join(data['Genre']).split(",")
    #print (categories)
    
    """on enlève les catégories débiles"""
    for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008','Hulu','Prime Video','fuboTV']:
        while i in categories:
            categories.remove(i)
    return(categories)

In [426]:
"""-------------------------Fonction qui retourne les effectifs de chaque genre-------------------------"""

def count_genra (data_f,nombre=7):
    
    categories=recup_cat(data_f)
    #print(categories)

    """On compte le nombre d'occurences de chaque type et on garde les 50 qui apparaissent le plus souvent"""
    counter_list = co.Counter(categories).most_common(nombre)
    #print ((counter_list))

    """On conserve le premier élément de chaque couple de counter_list (ie. sur le genre) et on inverse l'ordre de la liste"""
    labels = [_[0] for _ in counter_list][::-1]

    """Idem pour le nombre d'occurrences"""
    values = [_[1] for _ in counter_list][::-1]
                                    
    return (values,labels)

In [410]:
"""-------------------------Fonction qui trace un graph des effectifs des genres-------------------------"""

def trace(values,labels,nombre=7,date="Netflix") :  
    colors =['#DF0101','#086A87'] *int(nombre/2) if (nombre%2 == 0) else ['#086A87']+(['#DF0101','#086A87']*int(nombre/2))
    trace1 = go.Bar(y=labels, x=values, orientation="h",marker_color=colors, name="TV Shows", marker=dict(color="#a678de"))

    data = [trace1]
    layout = go.Layout(template = 'presentation',margin=dict(l=160, r=80, t=80, b=50),
                       title="Contenu sur "+str(date), font_size= 15, legend=dict(x=0.1, y=1.1, orientation="h"))
    fig = go.Figure(data, layout=layout)
    fig.show()
    return

In [427]:
trace(count_genra(AS_N)[0],count_genra(AS_N)[1])

On veut observer cette répartition pour une année donnée puis, à terme, comparer entre plusieurs années.
Pour cela on utilise:
- `liste_date` qui contient toutes les années où une série est *sortie*.

- `count_trace_genra` qui utilise les précédentes `count_genra` et `trace` pour tracer le graph batons d'une année donnée.

In [399]:
"""-------------------------Liste des dates où une ou plusieurs séries ont été crées-------------------------"""
liste_date=[]
for i in range(1969,2021):
    if AS_N['Year Released'].isin([i]).sum() != 0:
        liste_date.append(i)
print (liste_date)

[1969, 1972, 1983, 1986, 1987, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [412]:
"""---------------Fonction qui compte les effectifs des genres puis les représente pour une année donnée---------------"""

def count_trace_genra_y (annee,nombre=15):
    
    """On crée le df dont on a besoin à partir de la date voulue"""
    data_f = AS_N[AS_N['Year Released'].isin([annee])]
    
    a=count_genra(data_f,nombre)
    trace(a[0],a[1],nombre,annee)

    return

In [413]:
count_trace_genra_y(2020,10)

Pour comparer plusieurs années sur une même figure on va utiliser:
- `prep` une fonction intermédiaire qui retourne les effectifs des genres les plus populaires pour une année donnée
- `genra_comp` qui trace les effectifs des genres les plus populaires pour trois années données

In [416]:
"""----------Fonction intermédiaire qui compte les effectifs des genres les plus populaires pour une année donnée----------"""

def prep (annee,nombre=5):
    
    """On récupère les x genres les plus populaires overall """
    counter_list = co.Counter(recup_cat(AS_N)).most_common(nombre)
    counter_list = dict(counter_list)
    #print(counter_list)
    
    """On crée le df dont on a besoin à partir de la date voulue"""
    data_f = AS_N[AS_N['Year Released'].isin([annee])]
    
    """On récupère les genres les plus populaires de cette année """    
    categories_y=recup_cat(data_f)
    
    counter_list_y = co.Counter(categories_y).most_common()
    counter_list_y = dict(counter_list_y)
    
    dico=dict()
    for i in counter_list_y.keys():
        if i in counter_list:
            dico[i]=counter_list_y[i]
    
    """On récupère la liste des clés et celle des valeurs pour aléger l'écriture par la suite"""
    labels = list(dico.keys())
    values = list(dico.values())
    
    return (values,labels)

In [435]:
"""----------Fonction qui compare les effectifs des genres populaires pour 3 années données----------"""

def genra_comp(year1,year2,year3,nombre=5):
    fig = go.Figure(data=[
        go.Bar(name=str(year1), y=prep(year1,nombre)[0], x=prep(year1,nombre)[1]),
        go.Bar(name=str(year2), y=prep(year2,nombre)[0], x=prep(year2,nombre)[1]),
        go.Bar(name=str(year3), y=prep(year3,nombre)[0], x=prep(year3,nombre)[1])
    ])
    # Change the bar mode
    fig.update_layout(barmode='group')
    
    fig.update_layout(template = 'presentation',
                      title='Contenu populaire en '+str(year1)+", "+str(year2)+" et "+str(year3),
                      font_size= 15, legend=dict(yanchor="top", y=0.99, xanchor="right", x=1))
    fig.show()
    return

In [436]:
genra_comp(2017,2018,2019)

### Comparaisons entre les plateformes
La richesse de la base `All Streaming Shows` (aka `AS`) nous permet de comparer le contenu de Netflix a celui de ses concurrents.
Nous avons choisi : Prime Video, Hulu Disney+ et fuboTV dont les contenus sont consignés dans `AS_P`, `AS_H`, `AS_D` et `AS_F`.

#### Ancienneté du contenu
On se demande d'abord si Netflix a un contenu différent de ses concurrents selon la date de sortie.

Pour cela on réutilise la même formule de la somme cumulée des effectifs en fonction des années de *sortie* pour chacune des plateformes.

On a choisi de ne pas regarder le contenu avant 1990 parce que quelques titres écrasaient la distribution à droite et rendaient le graph peu lisible.

In [499]:
"""----------On se limite aux années post 1990----------"""

start_year=1990
years=[]
for i in range (start_year,2021):
    years.append(i)
    
AS_N=AS_N[AS_N['Year Released'].isin(years)]
AS_P=AS_P[AS_P['Year Released'].isin(years)]
AS_H=AS_H[AS_H['Year Released'].isin(years)]
AS_F=AS_F[AS_F['Year Released'].isin(years)]
AS_D=AS_D[AS_D['Year Released'].isin(years)]


"""----------Graphiques de l'ancienneté des séries sur les Plateformes----------"""

fig = plt.subplots.make_subplots(specs=[[{"secondary_y": True}]])
colors =['#DF0101','#086A87','#58FA82','yellow','purple']


for e,i in enumerate([AS_N,AS_P,AS_H,AS_F,AS_D]):
    
    somme_sorties=i['Year Released'].value_counts().sort_index().cumsum()
    trace = go.Scatter(x = list(somme_sorties.index), y = list(somme_sorties.values),
                       marker_color=colors[e],
                       mode='lines', name = ['Netflix','Prime','Hulu','fuboTV','Disney+'][e])
    fig.add_trace(trace, secondary_y=False)


fig.update_layout(template = 'presentation',margin=dict(l=80, r=80, t=25, b=50),
                  title = { 'text' : '<b>Ancienneté du contenu sur les plateformes</b>', 'x' : 0.5, 'y' : 0.94},
                 font_family = 'Calibri',font_size = 16, title_font_color= 'black', showlegend = True)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))

fig.show()

Il pourrait paraître étrange que Prime Vidéo ait pu avoir plus de contenu que Netflix pré-2019 mais ce serait justement oublier la nuance entre date de sortie et date de diffusion sur la plateforme !

En effet ce graphique nous dit seulement que Prime Video et Hulu on du contenu plus ancien que Netflix ce qui n'est pas aberrant puisqu'ils ont peu de contenu original et beaucoup de contenu racheté donc potentiellement sorti dans les années 2010 voir 2000.

Au contraire Netflix produit énormément de contenu original depuis quelques années ce qui explique qu'il aie beaucoup plus de contenu *sorti* en 2019, 2020 que ses concurrents.

On remarque par ailleurs que Netflix a beaucoup plus de contenu que ses concurrents fin 2020 : deux fois plus qu'Hulu et quatre fois plus que fuboTV.

#### Type du contenu
On se demande si Netflix s'est spécialisé dans des genres différents de ces concurrents?<br>
Pour cela on compare les x genres les plus présents sur chacune des palteformes.

In [531]:
"""----------On retire la restriction sur les dates----------"""
AS_N=AS[AS['Streaming Platform'].isin(['Netflix'])]
AS_P=AS[AS['Streaming Platform'].isin(['Prime Video'])]
AS_H=AS[AS['Streaming Platform'].isin(['Hulu'])]
AS_D=AS[AS['Streaming Platform'].isin(['Disney+'])]
AS_F=AS[AS['Streaming Platform'].isin(['fuboTV'])]

In [504]:
"""----------Fonction qui compare les genres les plus populaires en fonction des plateformes----------"""
def comp_platforms_genra(nombre):
    genre_count={}
    genre_count["Netflix"]=count_genra(AS_N,nombre)
    genre_count["Prime"]=count_genra(AS_P,nombre)
    genre_count["Hulu"]=count_genra(AS_H,nombre)
    genre_count["fuboTV"]=count_genra(AS_F,nombre)
    genre_count["Disney+"]=count_genra(AS_D,nombre)
    #print(genre_count)
    colors =['#DF0101','#086A87','#58FA82','yellow','purple']

    fig = plt.subplots.make_subplots(rows = 2, cols = 3, subplot_titles = ['Netflix','Prime','Hulu','fuboTV','Disney+'],
                                    horizontal_spacing=0.15, vertical_spacing=0.2)
    for e,i in enumerate(['Netflix','Prime','Hulu','fuboTV','Disney+']):
        trace = go.Bar(y = genre_count[i][1], x=genre_count[i][0],
                       marker_color=colors[e],name ="titres" , orientation = 'h')
        fig.add_trace(trace, row = (e//3) + 1, col = (e%3) + 1)

    fig.update_layout(showlegend = False)
    fig.update_layout(template = 'presentation',margin=dict(l=130, r=30, t=70, b=50),
                      title = { 'text' : '<b>Contenu le plus présent selon les plateformes</b>', 'x' : 0.5},
                     font_family = 'Calibri', font_size = 16, title_font_color= 'black', showlegend = False)
    fig.show()
    return
comp_platforms_genra(5)

Netflix ne semble pas vouloir se démarquer en se spécialisant dans un genre particulier, ces concurrents non plus d'ailleurs à une exception près. Cela n'est pas étonnant, Netflix n'a peut-être pas besoin de se spécialiser pour rester concurrentiel. <br>  Disney+ semble en revanche bien spécialisé malgré (ou à cause) son catalogue plus réduit.

#### Qualité du contenu
On se demande si Netflix obient des notes IMDB et Rotten Tomatoes comparables à celles de ses concurrents. <br>
Pour cela on :
- On concatène les cinq df vers `five`.
- On supprime les titres avec `R Rating`= -1, qui faussent la distribution et on obtient `five_clean`.
- On multiplie les notes IMDB par 10 pour pouvoir les afficher sur la même échelle que les notes Rotten Tomatoes.
- On trace les boîtes à moustaches des notes en fonction des Plateformes.

In [575]:
two=pd.concat([AS_N, AS_P])
three=pd.concat([two,AS_H])
four=pd.concat([three,AS_F])
five=pd.concat([four,AS_D])

In [577]:
five_clean=five
liste = []
for i in range(five[five['R Rating'].isin([-1])].shape[0]):
    liste.append(five[five['R Rating'].isin([-1])].index[i])
#print(liste)
for l in liste:
    #print (l)
    five_clean=five_clean.drop(l)

In [580]:
five_clean['R Rating'].isin([-1]).sum()

0

In [582]:
five_clean['IMDB Rating 10']=five_clean['IMDB Rating'].apply(lambda x: x*10)

x = five_clean['Streaming Platform']

fig = go.Figure()

fig.add_trace(go.Box(
    y=five_clean['IMDB Rating 10'],
    x=x,
    name='IMDB Rating',
    marker_color='#DBA506'
))
fig.add_trace(go.Box(
    y=five_clean['R Rating'],
    x=x,
    name='R Rating',
    marker_color='#DF0101'
))

fig.update_layout(
    template = 'presentation',
    title = "Notes en fontion de la Plateforme",
    yaxis_title='Notes',
    boxmode='group' # group together boxes of the different traces for each value of x
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=1.2,
    xanchor="left",
    x=0.01))

fig.show()

Sur IMDB les notes sont très similaires pour toutes les plateformes. En revanche les utilisateurs de Rotten Tomatoes semblent avoir une dent contre Prime Video et Netflix s'en sort avec de plutôt bons résultats.

# Modélisation

# Conclusion

# Annexe